# Unsupervised Outlier Detection on Heartbeat graphs

I am using Local Outlier Factor (LOF) method and k-nearest neighbors to do anomaly detection on the graph representation of the heart beat signal.

This notebook, is completing the final step of our pipeline and combining the steps and results of <code>MIT-BIH-Kaggle</code> notebook with <code>KNN_AnomalyDetection_OnGraphs</code> to have a complete process from a raw signal to detect anomalies for each heartbeat of the signal using its coresponding graph.

The outlies are the followings:
1. Create graph stream from the original signal.
2. Compute the pairwise distance between each two graphs and produce the distance matrix using Graph edit distance.
3. Feed the distance matrix for our graph stream into a Local Outlier Factor (LOF) model and detect the abnormal graphs (Each graph corespond to a heartbeat).
4. Compare the detected graph with the ground truth, The actual label of the heartbeat is labeled by expert Cardialogists, and compute the accuracy of our model.

## 1. Graph stream from the original signal.
Lets generate our graph stream from the **MIT-BIH** dataset using the helper functions which implemented  in <code>MIT-BIH-Kaggle</code>.

In [8]:
%matplotlib inline
from myHelper import timing
from myHelper import GraphSignalHelper

data_df = GraphSignalHelper.read_heartbeat_database(name='100', nRowsRead=3000)
annotations_df = GraphSignalHelper.read_annotation_database(name='100')

graph_stream, graph_stream_labels = GraphSignalHelper.generate_graph_stream(data_df, annotations_df, fs = 360)

# len(graph_stream) = 2272

# graph_stream_labels[1]['Type']
# 2    N
# 3    N

## 2. Calculate gragh edit distance matrix
Compute the pairwise distance between each two graphs and produce the distance matrix using Graph edit distance.

In [11]:
from myHelper import timing

# Gmatch4py use networkx graph 
import networkx as nx 
# import the GED using the munkres algorithm
import gmatch4py as gm


@timing.time_it
def GED_distance_matrix(graph_series):
    # convert from dictionary of graphs to list of graphs
    graph_stream_list = [v for k, v in graph_stream.items()] 
    ged=gm.GraphEditDistance(1,1,1,1) # all edit costs are equal to 1
    non_symetric_GED_matrix=ged.compare(graph_stream_list,None)
    symetric_ged_matrix = (non_symetric_GED_matrix.transpose()+non_symetric_GED_matrix)/2
    return symetric_ged_matrix

distance_matrix = GED_distance_matrix(graph_stream)
print("precomputed distance matrix =\n",distance_matrix)


GED_distance_matrix took 99325.33574104309   mil sec
precomputed distance matrix =
 [[   0.   552.  1036.  1021.  1038.   647.5 1612.5 1907.5 1040.5]
 [ 552.     0.  1011.  1003.  1011.   644.5 1584.  1920.  1012.5]
 [1036.  1011.     0.   540.   536.   987.  1518.5 2086.  1133.5]
 [1021.  1003.   540.     0.   528.   992.5 1352.  2062.5 1145.5]
 [1038.  1011.   536.   528.     0.   979.  1423.  2091.  1127.5]
 [ 647.5  644.5  987.   992.5  979.     0.  1602.  1914.5 1011. ]
 [1612.5 1584.  1518.5 1352.  1423.  1602.     0.  2684.  1725.5]
 [1907.5 1920.  2086.  2062.5 2091.  1914.5 2684.     0.  1838. ]
 [1040.5 1012.5 1133.5 1145.5 1127.5 1011.  1725.5 1838.     0. ]]


## 3. Anomaly detection on graphs using distance matrix
Feed the distance matrix for our graph stream into a Local Outlier Factor (LOF) model and detect the abnormal graphs (Each graph corespond to a heartbeat).

In [51]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=2,
                         algorithm="brute",
                         metric="precomputed",
#                          contamination=0.01, # the proportion of outliers in the data set float 
                         contamination='auto', # the proportion of outliers in the data set float 
                         novelty=False, # novelty=True if you want to use LOF for novelty 
                                        # detection and predict on new unseen data
                         n_jobs=None)

clf.fit(X=distance_matrix)


# predict on training data
# Fits the model to the training set X and returns the labels.
outlier_prediction = clf.fit_predict(X=distance_matrix)
print("outlier prediction (-1 = abnormal, 1 = normal) \n\n",
      outlier_prediction)

# Fit the model using X as training data.
clf.fit(X=distance_matrix)

# Observations having a negative_outlier_factor smaller than offset_ are detected as abnormal.
# The offset is set to -1.5 (inliers score around -1), except when
# a contamination parameter different than “auto” is provided.
print("\n Outlier threshold = ", -(clf.offset_))


# Only available for novelty detection (when novelty is set to True).
# The shift offset allows a zero threshold for being an outlier.
# print("decision_function = ", clf.decision_function(X=symetric_result))


#  negative_outlier_factor_ndarray shape (n_samples,)
#  negative_outlier_factor_ = -1 normal
#  negative_outlier_factor_ << -1 abnormal 
print("\n Outlier scores (1 = normal, 1 >> abnormal)= \n \n", -(clf.negative_outlier_factor_))

outlier prediction (-1 = abnormal, 1 = normal) 

 [ 1  1  1  1  1  1 -1 -1 -1]

 Outlier threshold =  1.5

 Outlier scores (1 = normal, 1 >> abnormal)= 
 
 [0.9988417  1.00232198 0.99814815 0.99814815 1.00371747 0.9988417
 2.57422036 2.37499727 1.56436237]


## 4. Evaluation to the real labels
Compare the detected anomaly graphs with the ground truth, The actual label of the heartbeat is labeled by expert Cardialogists, and compute the accuracy of our model.

In [54]:
for graph_index, graph_annotation in enumerate(graph_stream_labels):
    Label = np.where(graph_annotation["Type"] == "N", "Normal", "Abnormal")
    print("For graph ", graph_index, "  = ", Label)

For graph  0   =  ['Normal']
For graph  1   =  ['Normal' 'Normal']
For graph  2   =  ['Normal']
For graph  3   =  []
For graph  4   =  ['Normal']
For graph  5   =  ['Normal']
For graph  6   =  ['Normal' 'Abnormal']
For graph  7   =  ['Normal']
For graph  8   =  []


For the first 9 heartbeats it works and the result is acceptable. However, the problem is still the running time of the model.

If I run the code for all the rows of the dataset it will give me 2272 heartbeat. For this 2272 graphs, I have to compute the distance matrix of size $(2272*2272)$ which does not exceed my memory capacity but it will take 73 days to compute this distance matrix using <code>gmatch4py</code> algorithm.